## TF-IDF Feature Extraction and Random Forest Classifier

The following sources were used to construct this Jupyter Notebook:

* [Numpy: Dot Multiplication, Vstack, Hstack, Flatten](https://www.youtube.com/watch?v=nkO6bmp511M)
* [Scikit Learn TF-IDF Feature Extraction and Latent Semantic Analysis](https://www.youtube.com/watch?v=BJ0MnawUpaU)
* [Fake News Challenge TF-IDF Baseline](https://github.com/gmyrianthous/fakenewschallenge/blob/master/baseline.py)
* [Python TF-IDF Algorithm Built From Scratch](https://www.youtube.com/watch?v=hXNbFNCgPfY)
* [Theory Behind TF-IDF](https://www.youtube.com/watch?v=4vT4fzjkGCQ)

In [1]:
#Import all required modules
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import score

In [2]:
#Import data from CSV file and create a dataframe
def create_dataframe(filename):
    #Read file into a pandas dataframe
    df = pd.read_csv(filename)
    #Remove white space in column names
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df

In [3]:
#Create dataframes for both training and testing sets
train_df_tmp = create_dataframe('train_stances.csv')
test_df_tmp = create_dataframe('competition_test_stances.csv')
train_bodies_df = create_dataframe('train_bodies.csv')
test_bodies_df = create_dataframe('test_bodies.csv')

train_df_tmp.head(5)

,Headline,Body_ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [4]:
train_df = pd.merge(train_df_tmp,
                 train_bodies_df[['Body_ID', 'articleBody']],
                 on='Body_ID')

test_df = pd.merge(test_df_tmp,
                 test_bodies_df[['Body_ID', 'articleBody']],
                 on='Body_ID')

train_df = train_df.rename(columns={'articleBody': 'Body_Text'})
test_df = test_df.rename(columns={'articleBody': 'Body_Text'})

In [5]:
test_df.sort_values(by=['Body_ID']).head(5)

,Headline,Body_ID,Stance,Body_Text
7305,Apple to keep gold Watch Editions in special i...,1,unrelated,Al-Sisi has denied Israeli reports stating tha...
7303,Apple installing safes in-store to protect gol...,1,unrelated,Al-Sisi has denied Israeli reports stating tha...
7304,El-Sisi denies claims he'll give Sinai land to...,1,agree,Al-Sisi has denied Israeli reports stating tha...
7306,Apple Stores to Keep Gold “Edition” Apple Watc...,1,unrelated,Al-Sisi has denied Israeli reports stating tha...
7307,South Korean woman's hair 'eaten' by robot vac...,1,unrelated,Al-Sisi has denied Israeli reports stating tha...


In [6]:
train_df.sort_values(by=['Body_ID']).head(5)

,Headline,Body_ID,Stance,Body_Text
41651,"Soldier shot, Parliament locked down after gun...",0,unrelated,A small meteorite crashed into a wooded area i...
41657,Italian catches huge wels catfish; is it a rec...,0,unrelated,A small meteorite crashed into a wooded area i...
41658,Not coming to a store near you: The pumpkin sp...,0,unrelated,A small meteorite crashed into a wooded area i...
41659,One gunman killed in shooting on Parliament Hi...,0,unrelated,A small meteorite crashed into a wooded area i...
41660,Soldier shot at war memorial in Canada,0,unrelated,A small meteorite crashed into a wooded area i...


In [7]:
#Apply Scikit Learn TFIDF Feature Extraction Algorithm
body_text_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, stop_words='english',max_features=1024)
headline_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, stop_words='english',max_features=1024)

#Create vocabulary based on training data
train_body_tfidf = body_text_vectorizer.fit_transform(train_df['Body_Text'])
train_headline_tfidf = headline_vectorizer.fit_transform(train_df['Headline'])

#Use vocabulary for testing data
test_body_tfidf = body_text_vectorizer.transform(test_df['Body_Text'])
test_headline_tfidf = headline_vectorizer.transform(test_df['Headline'])

In [8]:
train_body_tfidf[0]

<1x1024 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [9]:
train_headline_tfidf[0]

<1x1024 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [10]:
#Tuple represents (Instance, Feature); value to the right of the tuple 
#represents the feature's tf-idf score
print(train_body_tfidf)

  (0, 332)	0.21723442952
  (0, 689)	0.327586484501
  (0, 78)	0.266411953695
  (0, 347)	0.117236954372
  (0, 870)	0.231179450995
  (0, 471)	0.600712680215
  (0, 116)	0.0818153766251
  (0, 136)	0.135276771531
  (0, 840)	0.128096663328
  (0, 741)	0.083971649992
  (0, 1014)	0.0726899751027
  (0, 622)	0.122306761113
  (0, 198)	0.0948914348433
  (0, 645)	0.0771207437796
  (0, 640)	0.117616840972
  (0, 562)	0.103367495456
  (0, 806)	0.0880628752502
  (0, 900)	0.11441287416
  (0, 227)	0.112455408311
  (0, 190)	0.114425967716
  (0, 111)	0.127989074129
  (0, 37)	0.116365810283
  (0, 248)	0.114191234884
  (0, 843)	0.085671211012
  (0, 784)	0.207991598627
  :	:
  (49971, 278)	0.0393187527713
  (49971, 39)	0.0387274273014
  (49971, 305)	0.0473208996525
  (49971, 978)	0.276515880077
  (49971, 877)	0.155937992539
  (49971, 69)	0.0441325314436
  (49971, 555)	0.0733436170549
  (49971, 81)	0.0499885476879
  (49971, 124)	0.0563253043726
  (49971, 529)	0.0434295088705
  (49971, 366)	0.0593902354402
  (499

In [11]:
#Merge headline and body_text tf-idf vectors together - Stack arrays in sequence horizontally
train_features = hstack([train_body_tfidf, train_headline_tfidf])
test_features = hstack([test_body_tfidf, test_headline_tfidf])

In [12]:
#Feature vector (the headline and body text features merged by hstack)
#SVD should be applied here?
print(train_features)

  (0, 332)	0.21723442952
  (0, 689)	0.327586484501
  (0, 78)	0.266411953695
  (0, 347)	0.117236954372
  (0, 870)	0.231179450995
  (0, 471)	0.600712680215
  (0, 116)	0.0818153766251
  (0, 136)	0.135276771531
  (0, 840)	0.128096663328
  (0, 741)	0.083971649992
  (0, 1014)	0.0726899751027
  (0, 622)	0.122306761113
  (0, 198)	0.0948914348433
  (0, 645)	0.0771207437796
  (0, 640)	0.117616840972
  (0, 562)	0.103367495456
  (0, 806)	0.0880628752502
  (0, 900)	0.11441287416
  (0, 227)	0.112455408311
  (0, 190)	0.114425967716
  (0, 111)	0.127989074129
  (0, 37)	0.116365810283
  (0, 248)	0.114191234884
  (0, 843)	0.085671211012
  (0, 784)	0.207991598627
  :	:
  (49966, 1282)	0.320908465616
  (49966, 1463)	0.37038430602
  (49966, 1435)	0.302270939807
  (49966, 1283)	0.377935337556
  (49966, 1038)	0.415450536405
  (49966, 1655)	0.426963209073
  (49967, 1078)	0.699266941339
  (49967, 1896)	0.714860647084
  (49968, 1790)	0.66558912334
  (49968, 1288)	0.746318376359
  (49969, 1222)	0.413619538917
  (

In [13]:
#Extract training and test labels
train_labels = list(train_df['Stance'])
test_labels = list(test_df['Stance'])

In [14]:
#Initialize random forest classifier (Scikit Learn)
rf_classifier = RandomForestClassifier(n_estimators=10)

y_pred = rf_classifier.fit(train_features, train_labels).predict(test_features)

accuracy_score(test_labels, y_pred)

0.64042025734860109

In [15]:
#Initialize multinomialnb classifier
nb_classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

y_pred = nb_classifier.fit(train_features,train_labels).predict(test_features)

accuracy_score(test_labels, y_pred)

0.70491480738204859

In [16]:
score.report_score(test_labels,y_pred)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    23     |     0     |    98     |   1782    |
-------------------------------------------------------------
| disagree  |     0     |     0     |     9     |    688    |
-------------------------------------------------------------
|  discuss  |    14     |     0     |    807    |   3643    |
-------------------------------------------------------------
| unrelated |    12     |     0     |   1253    |   17084   |
-------------------------------------------------------------
Score: 5131.25 out of 11651.25	(44.04033901941852%)


44.04033901941852

In [ ]:
#Add predicted labels to test dataframe
#test_df['RF_Predicted_Stance'] = list(y_pred)
#test_df2 = test_df[['Headline','Body_Text','RF_Predicted_Stance','Stance']]
#test_df2[test_df2['RF_Predicted_Stance'] == 'unrelated']